In [1]:
# Dependencies
from splinter import Browser
from bs4 import BeautifulSoup as bs
import pandas as pd
import requests
import pymongo
from flask import Flask, render_template, redirect
from flask_pymongo import PyMongo

In [2]:
executable_path = {'executable_path': '.\\resources\\chromedriver.exe'}
browser = Browser('chrome', **executable_path, headless=False)

In [55]:
## Data Scraping NASA's News Page ##

# Opens web page
nasa_url = 'https://mars.nasa.gov/news/?page=0&per_page=40&order=publish_date+desc%2Ccreated_at+desc&search=&category=19%2C165%2C184%2C204&blank_scope=Latest'

browser.visit(nasa_url)
html = browser.html

nasa_soup = bs(html, 'html.parser')

In [10]:
# Find the Latest News Headline 
latest_headline = nasa_soup.find_all('div',class_ = 'content_title')[1].text
headline_desc = nasa_soup.find_all('div', class_ = 'article_teaser_body')[0].text

print("Latest Headline:")
print(latest_headline)
print("Headline Description:")
print(headline_desc)

Latest Headline:
NASA Engineers Checking InSight's Weather Sensors
Headline Description:
An electronics issue is suspected to be preventing the sensors from sharing their data about Mars weather with the spacecraft.


In [56]:
## Data Scraping JPL's Space Image Page ##

# Find the JPL Featured Image
jpl_base_url = 'https://www.jpl.nasa.gov'
jpl_home_url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(jpl_home_url)

In [57]:
# Navigate to Full Image Url
browser.click_link_by_id(id = 'full_image')
browser.click_link_by_partial_text('more info')

html = browser.html
jpl_soup = bs(html, 'html.parser')

full_image_path = jpl_soup.find(class_ = 'main_image')["src"]

featured_image_url = jpl_base_url + full_image_path
print(featured_image_url)
browser.visit(featured_image_url)


https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA16029_hires.jpg


In [58]:
## Data Scraping Mars Facts ##

facts_url = "https://space-facts.com/mars/"

browser.visit(facts_url)
html = browser.html

facts_soup = bs(html, 'html.parser')


In [66]:
# Use pandas to scrape and mars facts
facts_table = pd.read_html(facts_url)[0]
facts_table

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers


In [75]:
# Convert the data to a html table string and save to 'mars_facts.html' for easy reference
facts_html = facts_table.to_html('mars_facts.html')


In [32]:
## Data Scraping Mars Hemisphere Images ##
hemi_base_url = "https://astrogeology.usgs.gov"
hemi_home_url = "https://astrogeology.usgs.gov/search/results?q=hemisphere+enhanced&k1=target&v1=Mars"

browser.visit(hemi_home_url)
html = browser.html

hemi_soup = bs(html, 'html.parser')


In [34]:
# Save HD image url string and corresponding titles
all_hemi_sections = hemi_soup.find_all(class_ = 'description')
all_hemi_sections

hemi_list = []

for hemi in all_hemi_sections:
    
    # Find Titles for Hemishperes
    hemi_title = hemi.find('h3').text
    
    # Navigate to next page urls
    hemi_nextpage = hemi.find('a')['href']
    browser.visit(hemi_base_url + hemi_nextpage)

    html2 = browser.html
    image_soup = bs(html2, 'html.parser')

    # Finds the HD image urls
    image_section = image_soup.find(class_ = 'downloads')
    hemi_image = image_section.find('li').a['href']

    # Stores results to a dict so we can append 'hemi_list'
    hemi_dict = {}
    hemi_dict['Hemisphere Name'] = hemi_title
    hemi_dict['Image Url'] = hemi_image

    hemi_list.append(hemi_dict)


print(hemi_list)


[{'Hemisphere Name': 'Cerberus Hemisphere Enhanced', 'Image Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/cerberus_enhanced.tif/full.jpg'}, {'Hemisphere Name': 'Schiaparelli Hemisphere Enhanced', 'Image Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/schiaparelli_enhanced.tif/full.jpg'}, {'Hemisphere Name': 'Syrtis Major Hemisphere Enhanced', 'Image Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/syrtis_major_enhanced.tif/full.jpg'}, {'Hemisphere Name': 'Valles Marineris Hemisphere Enhanced', 'Image Url': 'https://astropedia.astrogeology.usgs.gov/download/Mars/Viking/valles_marineris_enhanced.tif/full.jpg'}]
